# 0. Импорт библиотек

In [1]:
import numpy as np
import pandas as pd
#игнорирую предупреждение о производительности
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# 1. Импорт выгрузки бд

In [16]:
df=pd.read_excel('Vild_2023-10-17.xlsx')
df

,SUBJ_ID,SITE_ID,SUBJ_SCREEN,RAND_NUM,INIT,SUBJECT_STATUS,FINSTAT,BLOCK,DEL,V0_01_SVSTDAT,...,V3_12_HCG_LBORRES,V3_13_COSTAT,V3_13_COORRES,EOS_01_ENDAT,EOS_01_DECOD,EOS_01_LBL,EOS_01_REASON,EOS_01_REASONDESC,EOS_01_IEORRES,INVESTIGATOR_SIGNATURE
0,Код субъекта,Номер центра,Скрининговый номер,Рандомизационный код,Инициалы субъекта,Статус Субъекта,NaN,NaN,NaN,Дата визита,...,Результат,Есть комментарии к визиту?,Комментарии,Дата завершения участия в исследовании,Статус в конце исследования,"Пожалуйста, заполните «Визит досрочного заверш...",Причина досрочного прекращения участия в иссле...,"Пожалуйста, уточните",Если субъект выбыл из-за соответствия критерию...,Подпись исследователя
1,8455,02,01,R16,ДЖС,Завершил по протоколу,NaN,NaN,NaN,2023-09-21,...,NaN,NaN,NaN,2023-10-06,Завершил по протоколу,NaN,NaN,NaN,NaN,NaN
2,8456,02,02,R09,ЛПА,Завершил по протоколу,NaN,NaN,NaN,2023-09-21,...,NaN,NaN,NaN,2023-10-06,Завершил по протоколу,NaN,NaN,NaN,NaN,NaN
3,8457,02,03,R18,ВСА,Выбыл досрочно,NaN,NaN,NaN,2023-09-21,...,NaN,NaN,NaN,2023-09-27,Не закончил/прекратил,NaN,Соответствие критерию(ям) исключения,NaN,"✔ 11. Другие причины, которые, по мнению Иссле...",NaN
4,8458,02,04,R19,ОМП,Завершил по протоколу,NaN,NaN,NaN,2023-09-21,...,NaN,NaN,NaN,2023-10-06,Завершил по протоколу,NaN,NaN,NaN,NaN,NaN
5,8459,02,05,R04,СДС,Завершил по протоколу,NaN,NaN,NaN,2023-09-21,...,NaN,NaN,NaN,2023-10-06,Завершил по протоколу,NaN,NaN,NaN,NaN,NaN
6,8460,02,06,R05,РТД,Завершил по протоколу,NaN,NaN,NaN,2023-09-21,...,NaN,NaN,NaN,2023-10-06,Завершил по протоколу,NaN,NaN,NaN,NaN,NaN
7,8461,02,07,R17,ЩИВ,Завершил по протоколу,NaN,NaN,NaN,2023-09-22,...,NaN,NaN,NaN,2023-10-06,Завершил по протоколу,NaN,NaN,NaN,NaN,NaN
8,8462,02,08,R20,ВД,Завершил по протоколу,NaN,NaN,NaN,2023-09-22,...,NaN,NaN,NaN,2023-10-06,Завершил по протоколу,NaN,NaN,NaN,NaN,NaN
9,8463,02,09,R02,СЕА,Завершил по протоколу,NaN,NaN,NaN,2023-09-22,...,NaN,NaN,NaN,2023-10-06,Завершил по протоколу,NaN,NaN,NaN,NaN,NaN


In [17]:
Dict = df[:1].reset_index(drop = True).to_dict('list')

In [18]:
#Удаляем первую строку
df.drop (index=df.index [0], axis= 0 , inplace= True)

In [19]:
#список всех индексов в бд
listIN=list(df.columns)

Создание датасета с квери:

In [20]:
df_Q = pd.DataFrame(columns=['Скрининговый_номер', 'Идентификатор', 'Текст_квери'])
df_Q

,Скрининговый_номер,Идентификатор,Текст_квери


Подфункция для добавления строк в квери - датасет 

In [21]:
def Data_Query(query,ID,text):
    new_rows = pd.DataFrame({'Скрининговый_номер': query['SUBJ_SCREEN'].tolist(),
 'Идентификатор': [ID for i in range(len(query['SUBJ_SCREEN'].tolist()))],
 'Текст_квери': [text for i in range(len(query['SUBJ_SCREEN'].tolist()))]})
    return new_rows

Создание датасета с проверками:

In [22]:
df_Ch = pd.DataFrame(index=range(0, df.shape[0]+1))

In [23]:
def Data_Check(query,ID,data):
    data[ID] = 'Проверено'
    data.loc[:,ID].iloc[query.index.tolist(),] = 'Требует уточнения специалиста по обработке данных'

Создание датасета для уточнения специалиста по обработке данных:

In [24]:
df_M = pd.DataFrame(columns=['Скрининговый_номер', 'Идентификатор', 'Заголовок_в_эИРК','Данные_в_поле','Возможная_ошибка'])

In [25]:
def Data_Management(query,ID,text):
    new_rows_M = pd.DataFrame({'Скрининговый_номер': query['SUBJ_SCREEN'].tolist(),
 'Идентификатор': [ID for i in range(len(query['SUBJ_SCREEN'].tolist()))],
 'Заголовок_в_эИРК': [Dict.get(ID) for i in range(len(query['SUBJ_SCREEN'].tolist()))],
 'Данные_в_поле': query[ID].to_list(),
 'Возможная_ошибка': [text for i in range(len(query['SUBJ_SCREEN'].tolist()))]})
    return new_rows_M

Проверка расы:

In [26]:
Race = list(filter(lambda x: x.endswith('RACE'), listIN))

query = df.loc[df[[Race[0]]].iloc[:,0]  != 'Европеоидная']

df_Q = pd.concat([df_Q, Data_Query(query, Race[0], text = 'Пожалуйста, уточните расовую принадлежность')], ignore_index=True)

df_Ch = pd.concat([df_Ch, Data_Check(query,Race[0],df_Ch)], ignore_index=True)

df_M = pd.concat([df_M, Data_Management(query, Race[0], text = 'Указана некорректная расовая принадлежность')], ignore_index=True)

Подфункция для проверки другой расы, если есть:

In [27]:
RaceOth = list(filter(lambda x: x.endswith('RACEOTH'), listIN))

query = df.loc[df[[RaceOth[0]]].iloc[:,0].notna()]

df_Q = pd.concat([df_Q, Data_Query(query, RaceOth[0], text = 'Пожалуйста, уточните расовую принадлежность')], ignore_index=True)

df_Ch = pd.concat([df_Ch, Data_Check(query,RaceOth[0],df_Ch)], ignore_index=True)

df_M = pd.concat([df_M, Data_Management(query, RaceOth[0], text = 'Указана некорректная расовая принадлежность')], ignore_index=True)

Проверка этнической принадлежности:

In [14]:
Ethnic = list(filter(lambda x: x.endswith('ETHNIC'), listIN))

query = df.loc[df[[Ethnic[0]]].iloc[:,0]  != 'Не латиноамериканец']

df_Q = pd.concat([df_Q, Data_Query(query, Ethnic[0], text = 'Пожалуйста, уточните этническую принадлежность')], ignore_index=True)

df_Ch = pd.concat([df_Ch, Data_Check(query,Ethnic[0],df_Ch)], ignore_index=True)

df_M = pd.concat([df_M, Data_Management(query, Ethnic[0], text = 'Указана некорректная этническая принадлежность')], ignore_index=True)

IndexError: list index out of range

Употребление наркотиков должно быть отрицательное у всех субъектов и поле с уточнение наркотиков должно быть пустым:

In [15]:
sud = list(filter(lambda x: x.endswith('DRUGS_SUDOSE'), listIN))

query = df.loc[df[[sud[0]]].iloc[:,0].notna()]

df_Q = pd.concat([df_Q, Data_Query(query, sud[0], text = 'Пожалуйста, уточните употребляет ли субъект наркотики')], ignore_index=True)

df_Ch = pd.concat([df_Ch, Data_Check(query,sud[0],df_Ch)], ignore_index=True)

df_M = pd.concat([df_M, Data_Management(query, sud[0], text = 'Указана некорректная информация об употреблении наркотиков')], ignore_index=True)

IndexError: list index out of range

Неcохраненый репродуктивный потенциал у женщин младше 50

In [28]:
age = list(filter(lambda x: x.endswith('AGE'), listIN))
sex = list(filter(lambda x: x.endswith('SEX'), listIN))
Repr = list(filter(lambda x: x.endswith('CHILDPOT'), listIN))

query = df.loc[(df[[sex[0]]].iloc[:,0]  != 'Мужской')&(df[[Repr[0]]].iloc[:,0]  != 'Да')&(df[[age[0]]].iloc[:,0].astype(int)  <= 50)]

df_Q = pd.concat([df_Q, Data_Query(query, Repr[0], text = 'Пожалуйста, уточните сохранён ли у данного субъекта репродуктивный потенциал')], ignore_index=True)

df_Ch = pd.concat([df_Ch, Data_Check(query,Repr[0],df_Ch)], ignore_index=True)

df_M = pd.concat([df_M, Data_Management(query, Repr[0], text = 'Указан неверный статус репродуктивного потенциала (женщина младше 50)')], ignore_index=True)

Cохранённый репродуктивный потенциал у женщин старше 50 лет:

In [29]:
query = df.loc[(df[[sex[0]]].iloc[:,0]  != 'Мужской')&(df[[Repr[0]]].iloc[:,0]  != 'Нет')&(df[[age[0]]].iloc[:,0].astype(int)  >= 50)]

df_Q = pd.concat([df_Q, Data_Query(query, Repr[0], text = 'Пожалуйста, уточните сохранён ли у данного субъекта репродуктивный потенциал')], ignore_index=True)

df_Ch = pd.concat([df_Ch, Data_Check(query,Repr[0],df_Ch)], ignore_index=True)

df_M = pd.concat([df_M, Data_Management(query, Repr[0], text = 'Указан неверный статус репродуктивного потенциала (женщина старше 50)')], ignore_index=True)

Проверка на проведение всех процедур:

In [30]:
Lbstat = list(filter(lambda x: x.endswith('LBSTAT'), listIN))
for i in range(len(Lbstat)):
    query = df.loc[(df[Lbstat[i]]==df[Lbstat[i]])&(df[Lbstat[i]] != 'Да')]
    df_Q = pd.concat([df_Q, Data_Query(query, Lbstat[i], text = 'Пожалуйста, уточните процедура действительно была не проведена?')], ignore_index=True)
    df_Ch = pd.concat([df_Ch, Data_Check(query,Lbstat[i],df_Ch)], ignore_index=True)
    df_M = pd.concat([df_M, Data_Management(query, Lbstat[i], text = 'Указано, что процедура была непроведена')], ignore_index=True)

Проверка Физикального осмотра. Всё должно быть в норме.

In [31]:
Peorres = list(filter(lambda x: x.endswith('PEORRES'), listIN))
#Отбраcываю все пустые столбцы и записываю из в проверенные
query = df.loc[:, Peorres]
a = list(query.columns)
query.dropna(axis='columns',how='all', inplace=True)
Peorres = list(query.columns)
c = list(set(a) - set(Peorres))
for i in range(len(c)):
    df_Ch[c[i]] = 'Проверено'

In [32]:
#Отбираем все непустые поля с ненормой
for i in range(len(Peorres)):
    query = df.loc[(df[Peorres[i]]==df[Peorres[i]])&(df[Peorres[i]] != 'Норма')]
    df_Q = pd.concat([df_Q, Data_Query(query, Peorres[i], text = 'Пожалуйста, уточните верно ли указан клинический результат Физикального осмотра')], ignore_index=True)
    df_Ch = pd.concat([df_Ch, Data_Check(query,Peorres[i],df_Ch)], ignore_index=True)
    df_M = pd.concat([df_M, Data_Management(query, Peorres[i], text = 'Указан неверный клинический результат Физикального осмотра')], ignore_index=True)

Проверка комментариев после визитов:

In [33]:
coor = list(filter(lambda x: x.endswith('COORRES'), listIN))

for i in range(len(coor)):
    query = df.loc[df[[coor[i]]].iloc[:,0].notna()]
    df_Q = pd.concat([df_Q, Data_Query(query, coor[i], text = 'Пожалуйста, уточните комментарий к визиту')], ignore_index=True)
    df_Ch = pd.concat([df_Ch, Data_Check(query,coor[i],df_Ch)], ignore_index=True)
    df_M = pd.concat([df_M, Data_Management(query, coor[i], text = 'Указан некорректный комментарий к визиту')], ignore_index=True)

Комментарии должны быть пустые:

In [34]:
desc = list(filter(lambda x: x.endswith('DESC'), listIN))

for i in range(len(desc)):
    query = df.loc[df[[desc[i]]].iloc[:,0].notna()]
    df_Q = pd.concat([df_Q, Data_Query(query, desc[i], text = 'Пожалуйста, уточните комментарий')], ignore_index=True)
    df_Ch = pd.concat([df_Ch, Data_Check(query,desc[i],df_Ch)], ignore_index=True)
    df_M = pd.concat([df_M, Data_Management(query, desc[i], text = 'Указан некорректный комментарий')], ignore_index=True)

Проверка причин:

In [35]:
reasnd = list(filter(lambda x: x.endswith('REASND'), listIN))

for i in range(len(reasnd)):
    query = df.loc[df[[reasnd[i]]].iloc[:,0].notna()]
    df_Q = pd.concat([df_Q, Data_Query(query, reasnd[i], text = 'Пожалуйста, уточните причину непроведения')], ignore_index=True)
    df_Ch = pd.concat([df_Ch, Data_Check(query,reasnd[i],df_Ch)], ignore_index=True)
    df_M = pd.concat([df_M, Data_Management(query, reasnd[i], text = 'Указан некорректная причина непроведения')], ignore_index=True)

Проверка причины непроведения Анализа мочи:

In [36]:
reasnd_ur = list(filter(lambda x: x.endswith('LBREASND_UR'), listIN))

for i in range(len(reasnd_ur)):
    query = df.loc[df[[reasnd_ur[i]]].iloc[:,0].notna()]
    df_Q = pd.concat([df_Q, Data_Query(query, reasnd_ur[i], text = 'Пожалуйста, уточните причину непроведения общего анализа мочи')], ignore_index=True)
    df_Ch = pd.concat([df_Ch, Data_Check(query,reasnd_ur[i],df_Ch)], ignore_index=True)
    df_M = pd.concat([df_M, Data_Management(query, reasnd_ur[i], text = 'Указан некорректная причина непроведения общего анализа мочи')], ignore_index=True)

Проверка отсутствия причин непроведения анализа осадка мочи

In [37]:
reasnd_mi = list(filter(lambda x: x.endswith('LBREASND_MI'), listIN))

for i in range(len(reasnd_mi)):
    query = df.loc[df[[reasnd_mi[i]]].iloc[:,0].notna()]
    df_Q = pd.concat([df_Q, Data_Query(query, reasnd_mi[i], text = 'Пожалуйста, уточните причину непроведения анализа осадка мочи')], ignore_index=True)
    df_Ch = pd.concat([df_Ch, Data_Check(query,reasnd_mi[i],df_Ch)], ignore_index=True)
    df_M = pd.concat([df_M, Data_Management(query, reasnd_mi[i], text = 'Указана некорректная причина непроведения анализа осадка мочи')], ignore_index=True)

Проверка на отсутствие критериев исключения:

In [38]:
except_ieorres = list(filter(lambda x: x.endswith('EXCEPT_IEORRES'), listIN))

query = df.loc[df[[except_ieorres[0]]].iloc[:,0].notna()]

df_Q = pd.concat([df_Q, Data_Query(query, except_ieorres[0], text = 'Пожалуйста, уточните критерий исключения')], ignore_index=True)

df_Ch = pd.concat([df_Ch, Data_Check(query,except_ieorres[0],df_Ch)], ignore_index=True)

df_M = pd.concat([df_M, Data_Management(query, except_ieorres[0], text = 'Указана некорректная информация о критерии исключения')], ignore_index=True)

IndexError: list index out of range

Отбор крови на фармакокинетику:

In [39]:
pcdev = list(filter(lambda x: x.endswith('PCDEV'), listIN))

for i in range(len(pcdev)):
    query = df.loc[df[[pcdev[i]]].iloc[:,0].notna()]
    df_Q = pd.concat([df_Q, Data_Query(query, pcdev[i], text = 'Пожалуйста, уточните причину взятия пробы не по протоколу')], ignore_index=True)
    df_Ch = pd.concat([df_Ch, Data_Check(query,pcdev[i],df_Ch)], ignore_index=True)
    df_M = pd.concat([df_M, Data_Management(query, pcdev[i], text = 'Указана некорректная причина взятия пробы не по протоколу')], ignore_index=True)

Положение тела при приёме (сравнивается с модой):

In [40]:
position = list(filter(lambda x: x.endswith('POSITION_EXORRES'), listIN))

for i in range(len(position)):
    query = df.loc[(df[position[0]]==df[position[0]])&(df[position[0]] != df[position[0]].mode()[0])]
    df_Q = pd.concat([df_Q, Data_Query(query, position[i], text = 'Пожалуйста, уточните верно ли указан клинический результат Физикального осмотра')], ignore_index=True)
    df_Ch = pd.concat([df_Ch, Data_Check(query,position[i],df_Ch)], ignore_index=True)
    df_M = pd.concat([df_M, Data_Management(query, position[i], text = 'Указан неверный клинический результат Физикального осмотра')], ignore_index=True)

Проверка отсутствия анамнеза в начале исследования:

In [41]:
Mhoccur = list(filter(lambda x: x.endswith('MHOCCUR'), listIN))

query = df.loc[df[[Mhoccur[0]]].iloc[:,0]  != 'Нет']

df_Q = pd.concat([df_Q, Data_Query(query, Mhoccur[0], text = 'Пожалуйста, уточните наличие сопутствующих заболеваний')], ignore_index=True)

df_Ch = pd.concat([df_Ch, Data_Check(query,Mhoccur[0],df_Ch)], ignore_index=True)

df_M = pd.concat([df_M, Data_Management(query, Mhoccur[0], text = 'Указано наличие сопутствующих заболеваний')], ignore_index=True)

Регистрация нежелательных явлений (все "Да" необходимо перепроверить):

In [42]:
Aeoccur = list(filter(lambda x: x.endswith('AEOCCUR'), listIN))

for i in range(len(Aeoccur)):
    query = df.loc[(df[Aeoccur[i]]==df[Aeoccur[i]])&(df[Aeoccur[i]] != 'Нет')]
    df_Q = pd.concat([df_Q, Data_Query(query, Aeoccur[i], text = 'Пожалуйста, уточните было ли зарегестрировано НЯ у данного субъекта')], ignore_index=True)
    df_Ch = pd.concat([df_Ch, Data_Check(query,Aeoccur[i],df_Ch)], ignore_index=True)
    df_M = pd.concat([df_M, Data_Management(query, Aeoccur[i], text = 'Указано, что было зарегестрировано НЯ')], ignore_index=True)

Регистрация изменений Сопутствующей терапии:

In [43]:
Cmoccur = list(filter(lambda x: x.endswith('CMOCCUR'), listIN))

for i in range(len(Cmoccur)):
    query = df.loc[(df[Cmoccur[i]]==df[Cmoccur[i]])&(df[Cmoccur[i]] != 'Нет')]
    df_Q = pd.concat([df_Q, Data_Query(query, Cmoccur[i], text = 'Пожалуйста, уточните было ли зарегистрировано изменение в сопутствующей терапии?')], ignore_index=True)
    df_Ch = pd.concat([df_Ch, Data_Check(query,Cmoccur[i],df_Ch)], ignore_index=True)
    df_M = pd.concat([df_M, Data_Management(query, Cmoccur[i], text = 'Указано, что было зарегистрировано изменение в сопутствующей терапии')], ignore_index=True)

Соблюдение ограничений согласно протоколу:

In [44]:
Ieyn = list(filter(lambda x: x.endswith('IEYN')&x.endswith('IEYN'), listIN))
Ieyn = list(filter(lambda x: (not x.startswith('V0')), Ieyn))

for i in range(len(Aeoccur)):
    query = df.loc[(df[Ieyn[i]]==df[Ieyn[i]])&(df[Ieyn[i]] != 'Да')]
    df_Q = pd.concat([df_Q, Data_Query(query, Ieyn[i], text = 'Пожалуйста, уточните было ли несоблюдение ограничений по протоколу?')], ignore_index=True)
    df_Ch = pd.concat([df_Ch, Data_Check(query,Ieyn[i],df_Ch)], ignore_index=True)
    df_M = pd.concat([df_M, Data_Management(query, Ieyn[i], text = 'Указано, что было зарегистрировано несоблюдение ограничений по протоколу')], ignore_index=True)

IndexError: list index out of range

Указаны другие причины завершения досрочно (поле единственное?):

In [45]:
Reasoth = list(filter(lambda x: x.endswith('REASOTH'), listIN))

for i in range(len(Reasoth)):
    query = df.loc[df[[Reasoth[i]]].iloc[:,0].notna()]
    df_Q = pd.concat([df_Q, Data_Query(query, Reasoth[i], text = 'Пожалуйста, уточните причину досрочного завершения участия субъекта в исследования')], ignore_index=True)
    df_Ch = pd.concat([df_Ch, Data_Check(query,Reasoth[i],df_Ch)], ignore_index=True)
    df_M = pd.concat([df_M, Data_Management(query, Reasoth[i], text = 'Указана некорректная причина досрочного завершения участия субъекта в исследования')], ignore_index=True)

Проверка наличия фразы "Нет данных":

In [ ]:
for i in range(len(listIN)):
    query = df.loc[(df[listIN[i]]==df[listIN[i]])&(df[listIN[i]] == 'Нет данных')]
    df_Q = pd.concat([df_Q, Data_Query(query, listIN[i], text = 'Пожалуйста, уточните внесенные данные')], ignore_index=True)
    df_Ch = pd.concat([df_Ch, Data_Check(query,listIN[i],df_Ch)], ignore_index=True)
    df_M = pd.concat([df_M, Data_Management(query, listIN[i], text = 'Отсутствуют данные в поле')], ignore_index=True)

Проверка соблюдения про приём пищи и т.д.:

In [ ]:
Mlorres = list(filter(lambda x: x.endswith('MLORRES'), listIN))

for i in range(len(Mlorres)):
    query = df.loc[(df[Mlorres[i]]==df[Mlorres[i]])&(df[Mlorres[i]] != 'Да')]
    df_Q = pd.concat([df_Q, Data_Query(query, Mlorres[i], text = 'Пожалуйста, уточните было ли нарушение режима питания?')], ignore_index=True)
    df_Ch = pd.concat([df_Ch, Data_Check(query,Mlorres[i],df_Ch)], ignore_index=True)
    df_M = pd.concat([df_M, Data_Management(query, Mlorres[i], text = 'Указано, что было зарегистрировано арушение режима питания')], ignore_index=True)

Сортировка получившихся датасетов:

In [ ]:
df_Ch = df_Ch[sorted(df_Ch.columns)]
df_Ch

df_M и df_Q предлагаю сортировать по субъектам после просмотра возможных выявленных ошибок.

Запись проверок в файл Эксель:

In [ ]:
writer = pd.ExcelWriter('Milana_Project_Check_Visit.xlsx')

In [ ]:
df_Ch.to_excel(writer, 'CHECK', index= False)
df_M.to_excel(writer, 'Data_Manager', index= False)
df_Q.to_excel(writer, 'Query', index= False)

In [ ]:
writer.save()